# Домашнее задание к лекции "Основы веб-скрапинга и работы с API"¶

## Задание 1.
Обязательная часть
Будем парсить страницу со свежеми новостям на habr.com/ru/all/.

Вам необходимо собирать только те статьи, в которых встречается хотя бы одно требуемое ключевое слово. Эти слова определяем в начале кода в переменной, например:

KEYWORDS = ['python', 'парсинг']

Поиск вести по всей доступной preview-информации (это информация, доступная непосредственно с текущей страницы).

В итоге должен формироваться датафрейм со столбцами: <дата> - <заголовок> - <ссылка>.

Дополнительная часть (необязательная)
Улучшить скрипт так, чтобы он анализировал не только preview-информацию статьи, но и весь текст статьи целиком.

Для этого потребуется получать страницы статей и искать по тексту внутри этой страницы.

Итоговый датафрейм формировать со столбцами: <дата> - <заголовок> - <ссылка> - <текст статьи>

In [16]:
import requests
import time
from bs4 import BeautifulSoup
import pandas as pd

# указываю ключевые слова, по которым будет идти поиск
KEYWORDS = ['php', 'css', 'html', 'flex-box', 'java-script', 'sql']

# адрес для парсинга и подключение модулей + метод
request = requests.get('https://habr.com/ru/all/')
bs = BeautifulSoup(request.text, 'html.parser')
posts = bs.find_all('article', class_='post')

data = pd.DataFrame()
for post in posts:
    previews = post.find_all('div', "post__text_v1")
    date = post.find('span', "post__time").text
    title = post.find('h2', "post__title").text.strip(' \n')
    link = post.find('a', "post__title_link").get('href')
    for preview in previews:
        preview_lower = preview.text.lower()
    for keyword in KEYWORDS:
        if keyword in preview_lower:
            row = {'date': date, 'title': title, 'link': link}
            data = pd.concat([data, pd.DataFrame([row])])
            
data

,date,title,link
0,сегодня в 19:18,Пишем простую ORM с возможностью смены БД на лету,https://habr.com/ru/post/515068/
0,сегодня в 12:20,Дайджест интересных материалов для мобильного ...,https://habr.com/ru/company/digital-ecosystems...


In [17]:
import requests
import time
from bs4 import BeautifulSoup
import pandas as pd

# указываю ключевые слова, по которым будет идти поиск
KEYWORDS = ['php', 'css', 'html', 'flex-box', 'java-script', 'sql']

# адрес для парсинга и подключение модулей + метод
request = requests.get('https://habr.com/ru/all/')
bs = BeautifulSoup(request.text, 'html.parser')
posts = bs.find_all('article', class_='post')
data = pd.DataFrame()
links = []

for post in posts:
    link = post.find('a', "post__title_link").get('href')
    links.append(link)
for link in links:
    inner_request = requests.get(link)
    inner_bs = BeautifulSoup(inner_request.text, 'html.parser')
#    print(inner_soup)
    date = inner_bs.find('span', "post__time").text
    title = inner_bs.find('span', "post__title-text").text.strip()
    text = inner_bs.find('div', 'post__body post__body_full').text.strip()
#    print(text)
    text_lower = text.lower()
    for keyword in KEYWORDS:
        if keyword in text_lower:
            row = {'date': date, 'title': title, 'link': link, 'text':text[:60]}
            data = pd.concat([data, pd.DataFrame([row])])  

data

,date,title,link,text
0,сегодня в 21:47,FOSS News №30 – дайджест новостей свободного и...,https://habr.com/ru/post/516292/,Всем привет!\n\r\nПродолжаем дайджесты новосте...
0,сегодня в 21:17,Почему CDN не нужны: развенчиваем старый миф,https://habr.com/ru/post/516288/,Технология CDN была создана для ускорения разд...
0,сегодня в 20:19,Laravel–Дайджест (17–23 августа 2020),https://habr.com/ru/post/516240/,Перенос legacy-проекта на Laravel. Ускорение п...
0,сегодня в 20:19,Laravel–Дайджест (17–23 августа 2020),https://habr.com/ru/post/516240/,Перенос legacy-проекта на Laravel. Ускорение п...
0,сегодня в 19:18,Пишем простую ORM с возможностью смены БД на лету,https://habr.com/ru/post/515068/,"Привет, Хабр! Карма слита из-за неосторожного ..."
0,сегодня в 19:18,Пишем простую ORM с возможностью смены БД на лету,https://habr.com/ru/post/515068/,"Привет, Хабр! Карма слита из-за неосторожного ..."
0,сегодня в 17:37,О локальном поиске замолвите слово,https://habr.com/ru/post/516272/,В стародавние времена я работал айтишником в о...
0,сегодня в 16:16,Организация разработки крупномасштабных React-...,https://habr.com/ru/company/ruvds/blog/515682/,Эта публикация основана на серии материалов о ...
0,сегодня в 16:16,Организация разработки крупномасштабных React-...,https://habr.com/ru/company/ruvds/blog/515682/,Эта публикация основана на серии материалов о ...
0,сегодня в 16:00,AWS CLI через MFA,https://habr.com/ru/post/516262/,Далее последует инструкция по настройке AWS MF...


## Задание 2.

Обязательная часть Написать скрипт, который будет проверять список e-mail адресов на утечку при помощи сервиса Avast Hack Ckeck. Список email-ов задаем переменной в начале кода: EMAIL = [xxx@x.ru, yyy@y.com]

В итоге должен формироваться датафрейм со столбцами: <почта> - <дата утечки> - <источник утечки> - <описание утечки>.

Дополнительная часть (необязательная) Написать скрипт, который будет получать 50 последних постов указанной группы во Вконтакте. Документация к API VK: https://vk.com/dev/methods , вам поможет метод wall.getGROUP = 'netology' TOKEN = УДАЛЯЙТЕ В ВЕРСИИ ДЛЯ ПРОВЕРКИ, НА GITHUB НЕ ВЫКЛАДЫВАТЬ

В итоге должен формироваться датафрейм со столбцами: <дата поста> - <текст поста>.

ПРИМЕЧАНИЕ Домашнее задание сдается ссылкой на репозиторий GitHub. Не сможем проверить или помочь, если вы пришлете:

файлы; архивы; скриншоты кода. Все обсуждения и консультации по выполнению домашнего задания ведутся только на соответствующем канале в slack.

Как правильно задавать вопросы аспирантам, преподавателям и коллегам? Прежде чем задать вопрос необходимо попробовать найти ответ самому в интернете. Навык самостоятельного поиска информации – один из важнейших, и каждый практикующий специалист любого уровня это делает каждый день.

Любой вопрос должен быть сформулирован по алгоритму: 1) Что я делаю? 2) Какого результата я ожидаю? 3) Как фактический результат отличается от ожидаемого? 4) Что я уже попробовал сделать, чтобы исправить проблему?

По возможности, прикрепляйте к вопросу скриншоты, либо ссылки на код. Оставляйте только проблемный и воспроизводимый участок кода, все решение выкладывать не допускается.

In [36]:
import requests
import time
from bs4 import BeautifulSoup
import pandas as pd

EMAIL = ['awesomeemail@yahoo.com', 'test@test.com', 'helloOleg@gmail.com']

def leak(mail):
    URL = 'https://digibody.avast.com/v1/web/leaks'
    headers = {'Host': 'digibody.avast.com',
    'User-Agent': 'Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:79.0) Gecko/20100101 Firefox/79.0',
    'Accept': '*/*',
    'Accept-Language': 'en-US,en;q=0.5',
    'Accept-Encoding': 'gzip, deflate, br',
    'Access-Control-Request-Method': 'POST',
    'Access-Control-Request-Headers': 'content-type',
    'Referer': 'https://www.avast.com/hackcheck/',
    'Origin': 'https://www.avast.com',
    'Connection': 'keep-alive',
    'Cache-Control': 'max-age=0'}
    data = {
        'email': mail
    }
    request = requests.post(URL, json=data, headers=headers)
    df = pd.DataFrame()
    for leaks in request.json()['value']:
        user = leaks['username']
        date = int(leaks['leak_info']['date']/1000)
        source = leaks['leak_info']['title']
        description = leaks['leak_info']['description']
        row = {'username': user, 'date': date, 'title': source, 'description':description}
        df = pd.concat([df, pd.DataFrame([row])])
    return df

df_total = pd.DataFrame()
for email in EMAIL:
    df_total = pd.concat([df_total, leak(email)])

df_total['date'] = pd.to_datetime(df_total['date'], unit='s')
df_total.head(10)

,username,date,title,description
0,awesomeemail@yahoo.com,2017-12-22,Combolist of 1.4 Billion Credentials,The proliferation of stolen or leaked database...
0,awesomeemail@yahoo.com,2017-10-18,Anti-Public Combolist,The proliferation of stolen or leaked database...
0,awesomeemail@yahoo.com,2017-05-04,Active Account Cracking List,The proliferation of stolen or leaked-breach d...
0,awesomeemail@yahoo.com,2019-02-06,Collection #5 Combo List,"On January 7, 2019, an online user named Sanix..."
0,awesomeemail@yahoo.com,2019-02-06,2019 Antipublic Combo List,"On January 7, 2019, an online user named Sanix..."
0,awesomeemail,2017-12-22,Sensitive Source,This source has been marked as sensitive due t...
0,awesomeemail@yahoo.com,2019-01-29,Collection #2 Combo List,"On January 7, 2019, an online user named Sanix..."
0,punker6969,2016-10-21,MySpace,"Shortly before the 2016 Memorial Day weekend, ..."
0,awesomeemail@yahoo.com,2018-12-21,Sensitive Source,This source has been marked as sensitive due t...
0,-tribeofrainystorms-,2020-07-23,Wattpad,"In June 2020, the online writing community Wat..."
